<a href="https://colab.research.google.com/github/MahdiZaman/Spring2019-CAP5610/blob/master/hw1p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###HW1 P1

import numpy as np
from keras.datasets import mnist
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()


Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
# Preparing the Train and Test Images (Adding a column of 1s for biases)
nfeatures = 28*28
nclasses = 1
ntrainsamples = train_images_original.shape[0] #60000
ntestsamples = test_images_original.shape[0]   #10000

#normalizing inputs
train_images = train_images_original.reshape((ntrainsamples, nfeatures))
train_images = train_images.astype('float32') / 255 
test_images = test_images_original.reshape((ntestsamples, nfeatures))
test_images = test_images.astype('float32') / 255

#adding a column of 1s for bias calculation
train_images_b = np.c_[np.ones((ntrainsamples, 1)), train_images]
test_images_b = np.c_[np.ones((ntestsamples,1)), test_images]

# # One hot encoding for train and test labels
train_labels_bin = np.eye(10)[train_labels_original]
test_labels_bin = np.eye(10)[test_labels_original]

In [0]:
def sigmoid(z):
    a = 1/(1+ np.exp(-z))
    return a

In [0]:
def sigdif(a):
    ans = a *(1-a)
#     print("ans.shape=",ans.shape)
    return ans

In [0]:
def deriv(x, y, a, minibatch):
#     print("a.shape=",a.shape)
#     print("y.shape=",y.shape)
    temp = a-y
#     print("a-y.shape=",temp.shape)
    #print(".shape=",y.shape)
    dz = ((temp) * sigdif(a)) / minibatch
#     print("dz.shape=",dz.shape)
    dw = x.T.dot(dz)
#     dw = np.dot(x.T , dz)
    return dw

In [0]:
def forwardprop(x, weight):
    xw = x.dot(weight)
    A = sigmoid(xw)
    return A

In [0]:
def mse_loss(y, y_hat):
    l = 0.5*np.mean((y_hat-y)**2)
    return l

In [16]:
## Training the model

epochs = 20 #number of epochs
minibatchsize = 40 #number of samples in minibatch
lr = 0.01 #Learning Rate

#initializing Weights and Bias(Bias added as a column of 1 on the left in weight array)


for label in range(10):

    initial_weight = np.random.randn(nfeatures+1, nclasses)

    weight = initial_weight
  
#     weight_path_mgd = []
#     weight_path_mgd.append(weight)

    for epoch in range(epochs):
        #shuffling the samples
        m = np.random.permutation(ntrainsamples)
        train_images = train_images_b[m]
        train_labels = train_labels_bin[m]

        #minibatch SGD
        for i in range(0, ntrainsamples, minibatchsize):
            train_images_m = train_images[i : i+minibatchsize]
#             print(train_images_m.shape)
            train_labels_m = train_labels[i : i+minibatchsize]
            #print(train_labels_m.shape)
            A = forwardprop(train_images_m, weight)
#             print(weight.shape)

#             train_labels_l = train_labels_m[:,label]
#             print(train_labels_l.shape)
#             loss = mse_loss(train_labels_l, A))
#             if i ==0: print(A.shape) 
#             train_labels = train_labels_m == label
#             train_labels = train_labels_m == label #one-hot encoded train_labels
#             print(train_labels.shape)
#             print(A.shape)
#             break
#             print(train_labels[])
#             if i ==0 & epoch == 0: print(train_labels_m.shape)
#             if i ==0 & epoch == 0: print(train_labels_m[:,label])
            
            train_labels_l = train_labels_m[:,label]
            train_labels_l = train_labels_l.reshape((minibatchsize,1))
#             print(train_labels_l.shape)
#
#             print(train_labels_l.shape)
            loss = mse_loss(train_labels_l, A)
            
# # #             # Updating Weights and Biases
#             print(dw.shape)

#             print(train_images_m.shape)
#             print("A.shape", A.shape)
#             print("train_labels_l.ndim=",train_labels_l.ndim)
#             print("train_labels_l.shape=",train_labels_l.shape)
            
#             print(minibatchsize)

            dw = deriv(train_images_m, train_labels_l, A , minibatchsize)
            weight = weight - lr*dw
            
#             print(A.shape)

            #print("weight.shape=", weight.shape)
            #break
#             weight_path_mgd.append(weight)
     weight_path_mgd = np.array([])
     weight_path_mgd = np.c_[weight_path_mgd , weight.reshape((785,1))]
      
    print(weight_path_mgd.shape)
# weight_path_mgd.append(weight)
#        print(('Epoch {}/{} --- Training Loss {}'.format(epoch+1 , epochs, np.mean(loss))))
#     print(('Label {} Running, Loss = {}'.format(label,loss)))


ValueError: ignored

In [10]:
print(loss)
print(weight.shape)

0.049991013198969286
(785, 1)
